<a href="https://colab.research.google.com/github/tozanni/Data_Science_Notebooks/blob/main/CIFAR10_Autoencoder_CNN_BaselineReto_SinSolucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.utils import np_utils
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

Cargar el dataset cifar10 en un conjunto inicial de training (imagenes y labels) y test (imágenes y labels)

In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = datasets.cifar10.load_data()


In [ ]:
# Creamos la lista de etiquetas CIFAR10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### 1. Pre-procesamiento del dataset

a. Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

b. Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

c. Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


In [ ]:
# Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

# Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

# Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


d. Divide el test set en 7000 imagenes de validacion (x_val) y 3000 de test set (x_test)

In [ ]:
# Dividir test en validacion y test, 7000 de validacion y 3000 de test
x_val_images = x_test_images[:7000]
x_test_images = x_test_images[7000:]

e. Divide las etiquetas de validacion (y_val y y_test) de tal forma que correspondan a los conjuntos previos.

In [ ]:
# Modelo encoder-decoder de referencia

input_img = Input(shape=(32, 32, 3))

# Red encoder
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Red decoder
x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)


In [ ]:
# Modelo Encoder-Decoder
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        18464 

9. Entrenamiento del modelo no-supervisado encoder-decoder de tal forma que el modelo aprenda a reconstruir su propio input, con batch_size 64, mínimo 20 épocas y callbacks de EarlyStopping y ModelCheckpoint

10. Visualización del proceso de entrenamiento. Grafica Training Loss vs. Validation Loss.


## Visualización del input reconstruido

En este caso, el método predict de Keras entrega el input reconstruido.  Utiliza los siguientes métodos como referencia.

In [ ]:
c10test = model.predict(x_test_images)
c10val = model.predict(x_val_images)
print("Cifar10_test: {0}\nCifar10_val: {1}".format(np.average(c10test), np.average(c10val)))

In [ ]:
# Implementa una funcion para mostrar imagenes originales y reconstruidas

def showOrigDec(orig, dec, num=10):
    pass

In [ ]:
# Muestra algunas imagenes originales y reconstruidas utilizando tu funcion
#showOrigDec(x_test_images, c10test)


In [ ]:
# Modelo Encoder
encoder = Model(input_img, encoded)
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        1846

Extraer los features codificados para utilizar en un clasificador tipo MLP utilizando el método predict, del modelo encoder.

Utiliza el siguiente método como referencia.

In [ ]:

gist_train_ae = encoder.predict(x_train_images)
gist_valid_ae = encoder.predict(x_val_images)
gist_test_ae = encoder.predict(x_test_images)


Definir un clasificador con una capa convolucional y dos capas densas, que aprenderá a clasificar el input una vez pasado por el codificador. 

Puedes utilizar el siguiente clasificador como referencia:

In [ ]:
input = Input((gist_train_ae.shape[1], gist_train_ae.shape[2], gist_train_ae.shape[3]))

x = Conv2D(64, 3, padding="same")(input)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

decoder_classifier = Model(input, output)
decoder_classifier.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])
decoder_classifier.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4, 4, 16)]        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 4, 4, 64)          9280      
                                                                 
 activation_7 (Activation)   (None, 4, 4, 64)          0         
                                                                 
 batch_normalization_7 (Batc  (None, 4, 4, 64)         256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 64)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 2, 2, 64)          0   

Entrenamiento del clasificador MLP basado en la representación codificada, debes utilizar como input la representación codificada obtenida en el paso previo y como output los labels originales. 

Para validar el modelo, debes utilizar la representación codificada de los datos de validación.

Prediccion y evaluación del modelo de clasificación.

Utiliza el conjunto de test inicial, codifica lo con tu modelo encoder y obten una predicción con el modelo de clasificación. 


In [ ]:
# Predecir con el encoder
gist_test_ae = encoder.predict(x_test_images)

# Obtener las etiquetas con el clasificador
pred = decoder_classifier.predict(gist_test_ae)
print(pred)

# Converting the predictions into label index 
pred_classes = np.argmax(pred, axis=1)
print(pred_classes)


[[3.07077140e-01 8.79706442e-02 1.01494774e-01 ... 1.45255039e-02
  2.46723965e-01 7.89952800e-02]
 [1.50380190e-02 8.53455245e-01 1.40389928e-03 ... 3.07069859e-04
  2.72015762e-02 1.00546040e-01]
 [8.65967106e-03 1.52663710e-02 4.13899831e-02 ... 4.17461693e-01
  1.42259488e-03 4.61182557e-02]
 ...
 [9.16107558e-03 2.54500099e-03 1.25179678e-01 ... 5.98932356e-02
  4.15306864e-03 7.48984609e-03]
 [7.61062503e-02 4.52750027e-01 5.74691482e-02 ... 1.12841956e-01
  1.29908025e-02 1.37332529e-01]
 [3.65536548e-02 9.83946491e-03 7.11730197e-02 ... 5.61393976e-01
  5.96225960e-03 5.81768490e-02]]
[0 1 7 ... 5 1 7]


Generación de matriz de confusión (Baseline)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

cm=confusion_matrix(y_test_labels.argmax(axis=1), pred.argmax(axis=1))
print("Classification Report:\n")
cr=classification_report(y_test_labels.argmax(axis=1), pred.argmax(axis=1), target_names=class_names)
print(cr)

Classification Report:

              precision    recall  f1-score   support

    airplane       0.54      0.56      0.55       296
  automobile       0.50      0.64      0.56       311
        bird       0.41      0.27      0.32       299
         cat       0.33      0.35      0.34       302
        deer       0.51      0.28      0.37       278
         dog       0.45      0.31      0.37       293
        frog       0.55      0.57      0.56       302
       horse       0.58      0.53      0.55       298
        ship       0.59      0.68      0.64       314
       truck       0.43      0.65      0.51       307

    accuracy                           0.49      3000
   macro avg       0.49      0.48      0.48      3000
weighted avg       0.49      0.49      0.48      3000

